# Preprocess accessibility_coordinates.csv

**Imputation strategy:** Group median by `issue_type` + spatial bin
- For rows with missing severity: use median severity of same issue_type in same spatial bin
- Fallback: if bin has no data, use median by issue_type only
- Severity clipped to 1–4

In [6]:
import pandas as pd
import numpy as np

# Load data
df = pd.read_csv('/Users/devarshee/Desktop/Datathon/accessibility_coordinates.csv')
print(f"Shape: {df.shape}")
print(f"Missing severity: {df['severity'].isna().sum()}")
df.head()

Shape: (81973, 6)
Missing severity: 2251


,longitude,latitude,issue_type,severity,X,Y
0,-122.298981,47.594616,SurfaceProblem,4.0,1.278807e+06,220339.305108
1,-122.301071,47.593357,SurfaceProblem,3.0,1.278282e+06,219890.033975
2,-122.301079,47.596844,SurfaceProblem,4.0,1.278304e+06,221161.640591
3,-122.301071,47.596500,SurfaceProblem,4.0,1.278304e+06,221036.395318
4,-122.306274,47.599930,NoCurbRamp,4.0,1.277044e+06,222311.623280


In [7]:
# Create spatial bins (~1km grid: 0.01 degrees)
# Smaller bin = finer granularity, larger = more data per bin (less fallback)
BIN_SIZE = 0.01
df['lat_bin'] = (df['latitude'] // BIN_SIZE) * BIN_SIZE
df['lon_bin'] = (df['longitude'] // BIN_SIZE) * BIN_SIZE
df[['latitude', 'lat_bin', 'longitude', 'lon_bin']].head(10)

,latitude,lat_bin,longitude,lon_bin
0,47.594616,47.59,-122.298981,-122.30
1,47.593357,47.59,-122.301071,-122.31
2,47.596844,47.59,-122.301079,-122.31
3,47.596500,47.59,-122.301071,-122.31
4,47.599930,47.59,-122.306274,-122.31
5,47.600765,47.60,-122.310371,-122.32
6,47.595486,47.59,-122.300201,-122.31
7,47.600323,47.60,-122.300064,-122.31
8,47.600834,47.60,-122.300980,-122.31
9,47.601597,47.60,-122.298859,-122.30


In [8]:
# Compute median severity by (issue_type, lat_bin, lon_bin)
# Only use rows WITH severity to compute the median
median_by_spatial = df.groupby(['issue_type', 'lat_bin', 'lon_bin'])['severity'].transform(
    lambda x: x.dropna().median() if x.notna().any() else np.nan
)

# Fallback: median by issue_type only (when bin has no data)
median_by_issue_type = df.groupby('issue_type')['severity'].transform('median')

# Apply: first try spatial bin, then fallback to issue_type median
df['severity_imputed'] = df['severity'].fillna(median_by_spatial).fillna(median_by_issue_type)

# Replace original severity with imputed (keep imputed for rows that were missing)
df['severity'] = df['severity_imputed']
df = df.drop(columns=['severity_imputed'])

print(f"Missing severity after imputation: {df['severity'].isna().sum()}")

Missing severity after imputation: 253


In [9]:
# Clip severity to valid range (1-4) - handles outliers like 5.0
df['severity'] = df['severity'].clip(1, 4)
print("Severity distribution after imputation:")
print(df['severity'].value_counts().sort_index())

Severity distribution after imputation:
severity
1.0    17945
1.5       11
2.0    14899
2.5       18
3.0    23955
3.5       26
4.0    24866
Name: count, dtype: int64


In [ ]:
### Add population demographics

In [10]:
# Load Access dataset to get neighborhood for each (lon, lat)
access = pd.read_csv('/Users/devarshee/Desktop/Datathon/Data/Access_to_Everyday_Life_Dataset.csv')
access_coords = access[['geometry/coordinates/0', 'geometry/coordinates/1', 'properties/neighborhood']].copy()
access_coords.columns = ['longitude', 'latitude', 'neighborhood']

# Merge on coordinates (round to 6 decimals for matching)
df['lon_round'] = df['longitude'].round(6)
df['lat_round'] = df['latitude'].round(6)
access_coords['lon_round'] = access_coords['longitude'].round(6)
access_coords['lat_round'] = access_coords['latitude'].round(6)

# Keep one row per (lon_round, lat_round) if duplicates exist
access_unique = access_coords[['lon_round', 'lat_round', 'neighborhood']].drop_duplicates()
df = df.merge(access_unique, on=['lon_round', 'lat_round'], how='left')
df = df.drop(columns=['lon_round', 'lat_round'])
print(f"Rows with neighborhood: {df['neighborhood'].notna().sum()}")

Rows with neighborhood: 81973


In [11]:
# Load demographics and select key columns
demo = pd.read_csv('/Users/devarshee/Desktop/Datathon/Data/demographics_basic_age_sex_Neighborhoods.csv')
demo_sub = demo[['Neighborhood Name', 'Total', 'Children_under_18', 'Working_Age_Adults_18_64', 
                 'Older_Adults_65_over', 'Median Age Total']].copy()
demo_sub.columns = ['demo_neighborhood', 'population_total', 'children_under_18', 
                    'working_age_adults', 'older_adults_65_over', 'median_age']

# Map Access neighborhood to Demographics neighborhood
# Manual mapping for Access names that don't match Demographics "Neighborhood Name"
NEIGHBORHOOD_MAP = {
    'Atlantic': 'West Seattle Junction/Genesee Hill',
    'Adams': 'Columbia City',
    'Briarcliff': 'West Seattle Junction/Genesee Hill',
    'Broadway': 'First Hill/Capitol Hill',
    'Bryant': 'Ravenna/Bryant',
    'Central Business District': 'Downtown',
    'East Queen Anne': 'Queen Anne',
    'Eastlake': 'Cascade/Eastlake',
    'Harbor Island': 'Greater Duwamish',
    'Harrison/Denny-Blaine': 'Central Area/Squire Park',
    'Industrial District': 'Greater Duwamish',
    'Laurelhurst': 'Laurelhurst/Sand Point',
    'Lawton Park': 'Fauntleroy/Seaview',
    'Leschi': 'Madrona/Leschi',
    'Lower Queen Anne': 'Queen Anne',
    'Loyal Heights': 'Sunset Hill/Loyal Heights',
    'Madison Park': 'Madison Park',
    'Madrona': 'Madrona/Leschi',
    'Mann': 'North Beacon Hill/Jefferson Park',
    'Minor': 'Central Area/Squire Park',
    'Montlake': 'Montlake/Portage Bay',
    'Mount Baker': 'Mt. Baker/North Rainier',
    'North Beacon Hill': 'North Beacon Hill/Jefferson Park',
    'North Queen Anne': 'Queen Anne',
    'Phinney Ridge': 'Greenwood/Phinney Ridge',
    'Pike-Market': 'Downtown',
    'Pioneer Square': 'Pioneer Square/International District',
    'Portage Bay': 'Montlake/Portage Bay',
    'Ravenna': 'Ravenna/Bryant',
    'Roosevelt': 'Roosevelt',
    'Sand Point': 'Laurelhurst/Sand Point',
    'Southeast Magnolia': 'Magnolia',
    'Stevens': 'Central Area/Squire Park',
    'Sunset Hill': 'Sunset Hill/Loyal Heights',
    'University District': 'University District',
    'View Ridge': 'Wedgwood/View Ridge',
    'Wallingford': 'Wallingford',
    'West Queen Anne': 'Queen Anne',
    'West Woodland': 'Greenwood/Phinney Ridge',
    'Westlake': 'South Lake Union',
    'Whittier Heights': 'Whittier Heights',
    'Windermere': 'Northgate/Maple Leaf',
    'Yesler Terrace': 'Central Area/Squire Park',
}

def map_neighborhood(access_neigh):
    if pd.isna(access_neigh):
        return None
    access_neigh = str(access_neigh).strip()
    if access_neigh in NEIGHBORHOOD_MAP:
        return NEIGHBORHOOD_MAP[access_neigh]
    for demo_name in demo_sub['demo_neighborhood'].unique():
        if access_neigh in demo_name or access_neigh in demo_name.split('/'):
            return demo_name
    return None

df['demo_neighborhood'] = df['neighborhood'].apply(map_neighborhood)
match_rate = df['demo_neighborhood'].notna().mean() * 100
print(f"Matched to demographics: {match_rate:.1f}%")

Matched to demographics: 100.0%


In [12]:
# Join demographics (merge on mapped neighborhood)
df = df.merge(demo_sub, on='demo_neighborhood', how='left')

# Feature engineering: proportions and vulnerability index
df['pct_children'] = df['children_under_18'] / df['population_total'].replace(0, np.nan)
df['pct_older_adults'] = df['older_adults_65_over'] / df['population_total'].replace(0, np.nan)
df['vulnerability_index'] = df['pct_children'] + df['pct_older_adults']

# Fill missing demographics with median (for unmatched neighborhoods)
for col in ['population_total', 'children_under_18', 'working_age_adults', 'older_adults_65_over', 
            'median_age', 'pct_children', 'pct_older_adults', 'vulnerability_index']:
    df[col] = df[col].fillna(df[col].median())

# Drop helper columns
df = df.drop(columns=['neighborhood', 'demo_neighborhood'])

print("Demographic features added:")
print(df[['population_total', 'median_age', 'pct_children', 'pct_older_adults', 'vulnerability_index']].head())

Demographic features added:
   population_total  median_age  pct_children  pct_older_adults  \
0             23800        40.3      0.165504          0.130756   
1             23800        40.3      0.165504          0.130756   
2             23800        40.3      0.165504          0.130756   
3             23800        40.3      0.165504          0.130756   
4             23800        40.3      0.165504          0.130756   

   vulnerability_index  
0             0.296261  
1             0.296261  
2             0.296261  
3             0.296261  
4             0.296261  


### Save preprocessed data

In [14]:
# Drop spatial bin columns before saving (used only for imputation)
df_out = df.drop(columns=['lat_bin', 'lon_bin'])

# Save preprocessed data
output_path = '/Users/devarshee/Desktop/Datathon/accessibility_coordinates_preprocessed.csv'
df_out.to_csv(output_path, index=False)
print(f"Saved preprocessed data to {output_path}")
df_out.head()

Saved preprocessed data to /Users/devarshee/Desktop/Datathon/accessibility_coordinates_preprocessed.csv


,longitude,latitude,issue_type,severity,X,Y,population_total,children_under_18,working_age_adults,older_adults_65_over,median_age,pct_children,pct_older_adults,vulnerability_index
0,-122.298981,47.594616,SurfaceProblem,4.0,1.278807e+06,220339.305108,23800,3939,16749,3112,40.3,0.165504,0.130756,0.296261
1,-122.301071,47.593357,SurfaceProblem,3.0,1.278282e+06,219890.033975,23800,3939,16749,3112,40.3,0.165504,0.130756,0.296261
2,-122.301079,47.596844,SurfaceProblem,4.0,1.278304e+06,221161.640591,23800,3939,16749,3112,40.3,0.165504,0.130756,0.296261
3,-122.301071,47.596500,SurfaceProblem,4.0,1.278304e+06,221036.395318,23800,3939,16749,3112,40.3,0.165504,0.130756,0.296261
4,-122.306274,47.599930,NoCurbRamp,4.0,1.277044e+06,222311.623280,23800,3939,16749,3112,40.3,0.165504,0.130756,0.296261
